In [3]:
import numpy as np

# Test data

In [137]:
# Test data is structured as [time_series, is_trend]
test_data = [[[1,  3,  2, 1, 2, 4, 8, 10], True],
             [[1,  3,  2, 1, 2, 3, 1, 2],  False],
             [[1,  1,  1, 1, 1, 1, 1, 1],  False],
             [[10, 8,  4, 1, 1, 1, 1, 1],  False],
             [[20, 10, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  False],
             [[1,  1,  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  False],
             [[2,  1,  2, 3, 2, 3, 2, 1, 1, 3, 2, 1, 2, 2, 3, 2],  False],
             [[2,  1,  2, 3, 2, 3, 2, 1, 1, 3, 2, 1, 2, 4, 8, 10], True],
             [[0,  1,  0, 0, 577,124,54,13,10], False],
             [[0,  0,  0, 1, 0,0,673,224,78], False],
             [[0,  0,  0, 0, 0,0,265,51,19], False],
             [[0,  0,  0, 0, 265,51,16,86,29], False],
             [[0,  0,  0, 0, 0,0,91,86,28], False],
             [[0,  0,  0, 0, 0,167,50,11,27], False],
             [[0,  0,  0, 0, 0,40,17,11,4], False],
             [[0,  0,  0, 0, 0, 0, 0, 0 ,20], True],
             [[0,  0,  0, 0, 0, 0, 0, 0 ,30], True],
             [[0,  0,  0, 0, 0, 0, 0, 0 ,40], True],
             [[0,  0,  0, 0, 0, 0, 20,30,40], True],
             [[4,15,9,10,9,4,14,8,9], False],
             [[11,25,89,36,20,44,21,12,3], False],
             [[1,2,3,10,1,1,2,10,9], False],
             [[0,13,12,17,44,18,11,3,3], False],
             [[0,0,0,0,0,35,0,0,0], False],
             [[1,2,1,3,2,1,2,7,8], True],
             [[2,3,2,6,3,2,1,0,0], False],
             [[3,11,17,6,6,6,6,11,1], False],
             [[3,0,0,0,0,0,0,2,0], False],
             [[3,4,3,5,5,3,11,3,1], False],
             [[3,6,6,6,6,11,1,4,5], False],
             [[39,51,16,86,36,20,45,19,9], False],
             [[6,1,3,1,2,2,3,4,2], False],
             [[2,3,5,2,1,3,3,3,1], False],
             [[2,20,13,16,21,9,8,8,6], False],
             [[2,3,4,4,6,16,18,16,21], True],
             [[3,0,0,0,0,1,0,0,0], False],
             [[116,224,105,39,27,23,50,22,11], False],
             [[0,0,1,0,2,1,1,1,0], False]]

### Evaluation function

In [115]:
def evaluate_test_case(test_data, func, verbose=False):
    failed_list = []
    for test_case in test_data:
        failed = test_case[1] != func(test_case[0], verbose)
        if failed:
            failed_list += [test_case]
        print "Processing: "+ str(test_case)
        print "!!! F A I L E D !!!" if failed else "==> Success!!!"
    print "TOTAL FAILURES: "+ str(len(failed_list)) + "/" + str(len(test_data))
    return failed_list

# Home-made Approach 1

In [143]:
def home_made_approach_1(time_series, verbose=False):
    # a) Obtain mu as median
    mu = np.median(time_series)
    
    if verbose:
        print "a) mu: " + str(mu)

    # b) Only for values greater than the mu, indicate the difference
    time_series_diff = map(lambda t: 0.0 if t <= mu else (t - mu) , time_series)
    
    if verbose:
        print "b): " + str(time_series_diff)

    # c) Map to percentage values
    time_series_diff_perc = map(lambda t: round(float(t) / (mu + 0.01), 3), time_series_diff)
    
    if verbose:
        print "c): " + str(time_series_diff_perc)

    # d) Consider only those values more than 50% above mu
    time_series_diff_perc_filtered = map(lambda t: 0.0 if t <= 0.5 else t, time_series_diff_perc)
    
    if verbose:
        print "d): " + str(time_series_diff_perc_filtered)
        
    if sum(time_series_diff_perc_filtered) == 0:
        return False
        
    # e) Create a sorted version of time_series values
    sorted_time_series_diff_perc_filtered = sorted(time_series_diff_perc_filtered)

    if verbose:
        print "e): " + str(sorted_time_series_diff_perc_filtered)
        
    # f) Perform an element-wise subtraction between the list and the sorted version (e)
    element_wise_diff = map(lambda (i,t): 1.0 if max(t, sorted_time_series_diff_perc_filtered[i]) == 0 else 
                                          round(float(min(t, sorted_time_series_diff_perc_filtered[i]))/ 
                                                float(max(t, sorted_time_series_diff_perc_filtered[i])),3), 
                            enumerate(time_series_diff_perc))
    if verbose:
        print "f): " + str(element_wise_diff)

    # g) Give different weight in the diff of zeros
    element_wise_diff = map(lambda (i,t): 0.5 if (time_series_diff_perc_filtered[i] +
                                                  sorted_time_series_diff_perc_filtered[i]) == 0 else t,
                            enumerate(element_wise_diff))
    if verbose:
        print "g): " + str(element_wise_diff)

    # h) Calculate de mean of element_wise_diff. The closer to 1.0 the more likely to be trend
    mean_element_wise_diff = round(np.mean(element_wise_diff),3)
    
    if verbose:
        print "h): " + str(mean_element_wise_diff)

    return mean_element_wise_diff > 0.55

### Evaluating against test data

In [144]:
failed_cases = evaluate_test_case(test_data, home_made_approach_1, verbose=False)

Processing: [[1, 3, 2, 1, 2, 4, 8, 10], True]
==> Success!!!
Processing: [[1, 3, 2, 1, 2, 3, 1, 2], False]
==> Success!!!
Processing: [[1, 1, 1, 1, 1, 1, 1, 1], False]
==> Success!!!
Processing: [[10, 8, 4, 1, 1, 1, 1, 1], False]
==> Success!!!
Processing: [[20, 10, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], False]
==> Success!!!
Processing: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], False]
==> Success!!!
Processing: [[2, 1, 2, 3, 2, 3, 2, 1, 1, 3, 2, 1, 2, 2, 3, 2], False]
==> Success!!!
Processing: [[2, 1, 2, 3, 2, 3, 2, 1, 1, 3, 2, 1, 2, 4, 8, 10], True]
==> Success!!!
Processing: [[0, 1, 0, 0, 577, 124, 54, 13, 10], False]
==> Success!!!
Processing: [[0, 0, 0, 1, 0, 0, 673, 224, 78], False]
==> Success!!!
Processing: [[0, 0, 0, 0, 0, 0, 265, 51, 19], False]
==> Success!!!
Processing: [[0, 0, 0, 0, 265, 51, 16, 86, 29], False]
==> Success!!!
Processing: [[0, 0, 0, 0, 0, 0, 91, 86, 28], False]
==> Success!!!
Processing: [[0, 0, 0, 0, 0, 167, 50, 11, 27], False]
==> Success!!!

In [142]:
evaluate_test_case(failed_cases, home_made_approach_1, verbose=True)

TOTAL FAILURES: 0/0


[]